# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [74]:
# Import libraries
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import re

In [75]:
# Importing the dataset from Ironhack's database
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'pokemon'

connection_string = f"{driver}//{user}:{password}@{ip}/{database}"

engine = create_engine(connection_string)

In [76]:
query = """
SELECT *
FROM pokemon_stats
"""

pokemon = pd.read_sql(query, connection_string)
pokemon.head()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '35.239.232.23' (timed out)")
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# Rename columns for simplicity

pokemon.columns = ['#','Name','type_1','type_2','total','hp','attack','defense', 'sp_atk','sp_def','speed',
                   'generation','legendary']

pokemon.head()


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [ ]:
# your code here

pokemon['total_check']= pokemon.loc[:,['hp','attack','defense', 'sp_atk','sp_def','speed']].sum(axis=1)

pokemon.info()

pokemon[pokemon['total'] == pokemon['total_check']].info()

# The hypothesis seems to be correct, the total columns is the sum of all the attributes except generation and legendary

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [96]:
# your code here

#pokemon['type_1'] = pd.Categorical(pokemon['type_1'])
#pokemon['type_2'] = pd.Categorical(pokemon['type_2'])


dfDummies_type_1 = pd.get_dummies(pokemon['type_1'], prefix ='cat')
dfDummies_type_2 = pd.get_dummies(pokemon['type_2'], prefix ='cat')

pokemon_cat = (dfDummies_type_1 + dfDummies_type_2).drop(['cat_'], axis=1)
pokemon_cat.head()


#one_hot_pkmn_types = pokemon[[‘Type 1’, ‘Type 2’]].stack().str.get_dummies().sum(level=1)

,cat_Bug,cat_Dark,cat_Dragon,cat_Electric,cat_Fairy,cat_Fighting,cat_Fire,cat_Flying,cat_Ghost,cat_Grass,cat_Ground,cat_Ice,cat_Normal,cat_Poison,cat_Psychic,cat_Rock,cat_Steel,cat_Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [122]:
# your code here
pokemon_cat['total'] = pokemon['total']

pokemon_corr_total_types = pokemon_cat.corr(method='pearson').drop(pokemon_corr_total_types.iloc[:,:18], axis=1)
pokemon_corr_total_types = pokemon_corr_total_types.sort_values(by = 'total', ascending = False)


In [123]:
pokemon_corr_total_types

,total
total,1.000000
cat_Dragon,0.229705
cat_Psychic,0.124688
cat_Steel,0.109703
cat_Fire,0.078726
cat_Fighting,0.077786
cat_Ice,0.060248
cat_Flying,0.059383
cat_Dark,0.056154
cat_Rock,0.032731


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here

#I would choose a pokemon who is type1 Dragon and type 2 Psychic like Mega Latias